In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import re
import functools
from time import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import utils
import data_reader
from batch_norm import Batch_Normalizer
from model import CNN_Model

In [2]:
# File parameters
DATA_DIR = 'data'
TRAIN_IMAGES = 'data_train.bin'
TRAIN_TEMPLATES = 'fv_train.bin'
VALID_IMAGES = 'data_valid.bin'
VALID_TEMPLATES = 'fv_valid.bin'
TEST_IMAGES = 'data_test.bin'

# Data parameters
NUM_TRAIN_IMAGES = 100000
NUM_VALID_IMAGES = 10000
NUM_TEST_IMAGES = 10000
IMAGE_DIM = 48
TEMPLATE_DIM = 128

# Batch parameters
BATCH_SIZE = 100
VALID_BATCH_SIZE = 1000

# Batch norm parameters
EMA_DECAY = 0.99
BN_EPSILON = 0.01

# Log parameters
LOG_DIR = 'logs'
SAVE_DIR = 'checkpoints'

In [3]:
def run5():

    model = CNN_Model('config5', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
                      VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

    model.add_conv('conv_1_1', ksize=[3, 3, 1, 10])
    model.add_batch_norm('bn_1_1')
    model.add_conv('conv_1_2', ksize=[3, 3, 10, 10])
    model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_1_2')

    model.add_conv('conv_2_1', ksize=[3, 3, 10, 20])
    model.add_batch_norm('bn_2_1')
    model.add_conv('conv_2_2', ksize=[3, 3, 20, 20])
    model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_2_2')

    model.add_conv('conv_3_1', ksize=[3, 3, 20, 40])
    model.add_batch_norm('bn_3_1')
    model.add_conv('conv_3_2', ksize=[3, 3, 40, 40])
    model.add_batch_norm('bn_3_2')
    model.add_conv('conv_3_3', ksize=[3, 3, 40, 40])
    model.add_pool('max_pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_3_3')

    model.add_pool('avg_pool', 'avg', ksize=[1, 6, 6, 1], stride=[1, 6, 6, 1])
    model.add_fully_connected('fc', size=128)

    model.add_mse_loss('mse_loss')
    model.add_adam_optimizer('optimizer', init_learning_rate=0.005, decay=False)
    model.add_summaries('summaries')

    
#     utils.add_description("""
#     model = CNN_Model('config5', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
#                       VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

#     model.add_conv('conv_1_1', ksize=[3, 3, 1, 10])
#     model.add_batch_norm('bn_1_1')
#     model.add_conv('conv_1_2', ksize=[3, 3, 10, 10])
#     model.add_pool('max_pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_1_2')

#     model.add_conv('conv_2_1', ksize=[3, 3, 10, 20])
#     model.add_batch_norm('bn_2_1')
#     model.add_conv('conv_2_2', ksize=[3, 3, 20, 20])
#     model.add_pool('max_pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_2_2')

#     model.add_conv('conv_3_1', ksize=[3, 3, 20, 40])
#     model.add_batch_norm('bn_3_1')
#     model.add_conv('conv_3_2', ksize=[3, 3, 40, 40])
#     model.add_batch_norm('bn_3_2')
#     model.add_conv('conv_3_3', ksize=[3, 3, 40, 40])
#     model.add_pool('max_pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_3_3')

#     model.add_pool('avg_pool', 'avg', ksize=[1, 6, 6, 1], stride=[1, 6, 6, 1])
#     model.add_fully_connected('fc', size=128)

#     model.add_mse_loss('mse_loss')
#     model.add_adam_optimizer('optimizer', init_learning_rate=0.005, decay=False)
#     model.add_summaries('summaries')    

#     total parameters = %d
#     """  % (utils.num_parameters(model)), model.save_dir, model.name)
    
    
    model.initialize_session(restore=True)
    model.train(n_batches=10000, step_size=500, batch_size=100, valid_batch_size=1000, save=True)
    model.close_session()

In [4]:
def run1():

    model = CNN_Model('config1', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, 
                      TRAIN_TEMPLATES, VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

    model.add_conv('conv_1', ksize=[5, 5, 1, 32], padding='VALID')
    model.add_pool('pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_1')

    model.add_conv('conv_2', ksize=[3, 3, 32, 32], padding='VALID')
    model.add_pool('pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_2')

    model.add_conv('conv_3', ksize=[3, 3, 32, 32], padding='VALID')
    model.add_pool('pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_3')

    model.add_conv('conv_4', ksize=[3, 3, 32, 64], padding='VALID')
    model.add_pool('pool_4', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
    model.add_batch_norm('bn_4')

    model.add_fully_connected('fc', size=128)

    model.add_mse_loss('mse_loss')
    model.add_adam_optimizer('optimizer', init_learning_rate=0.01, decay=False)
    model.add_summaries('summaries')


    
#     utils.add_description("""
#     model = CNN_Model('config1', DATA_DIR, LOG_DIR, SAVE_DIR, IMAGE_DIM, TEMPLATE_DIM, TRAIN_IMAGES, TRAIN_TEMPLATES, 
#                       VALID_IMAGES, VALID_TEMPLATES, NUM_TRAIN_IMAGES, NUM_VALID_IMAGES, EMA_DECAY, BN_EPSILON)

#     model.add_conv('conv_1', ksize=[5, 5, 1, 32], padding='VALID')
#     model.add_pool('pool_1', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_1')

#     model.add_conv('conv_2', ksize=[3, 3, 32, 32], padding='VALID')
#     model.add_pool('pool_2', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_2')

#     model.add_conv('conv_3', ksize=[3, 3, 32, 32], padding='VALID')
#     model.add_pool('pool_3', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_3')

#     model.add_conv('conv_4', ksize=[3, 3, 32, 64], padding='VALID')
#     model.add_pool('pool_4', 'max', ksize=[1, 2, 2, 1], stride=[1, 2, 2, 1])
#     model.add_batch_norm('bn_4')

#     model.add_fully_connected('fc', size=128)

#     model.add_mse_loss('mse_loss')
#     model.add_adam_optimizer('optimizer', init_learning_rate=0.01, decay=False)
#     model.add_summaries('summaries')
    
#     total parameters = %d
#     """  % (utils.num_parameters(model)), model.save_dir, model.name)
    
    
    model.initialize_session(restore=True)
    model.train(n_batches=10000, step_size=500, batch_size=100, valid_batch_size=1000, save=True)
    model.close_session()

In [5]:
run5()

Restored session checkpoints\config5\model.ckpt-10500
Batch 11000:
--> Train loss = 0.006399
--> Valid loss = 0.007368
Saving session checkpoints\config5\model.ckpt-11000
Batch 11500:
--> Train loss = 0.006390
--> Valid loss = 0.007427
Saving session checkpoints\config5\model.ckpt-11500
Batch 12000:
--> Train loss = 0.006377
--> Valid loss = 0.007247
Saving session checkpoints\config5\model.ckpt-12000
Batch 12500:
--> Train loss = 0.006363
--> Valid loss = 0.007471
Saving session checkpoints\config5\model.ckpt-12500
Batch 13000:
--> Train loss = 0.006358
--> Valid loss = 0.007188
Saving session checkpoints\config5\model.ckpt-13000
Batch 13500:
--> Train loss = 0.006361
--> Valid loss = 0.007432
Saving session checkpoints\config5\model.ckpt-13500
Batch 14000:
--> Train loss = 0.006355
--> Valid loss = 0.007236
Saving session checkpoints\config5\model.ckpt-14000
Batch 14500:
--> Train loss = 0.006358
--> Valid loss = 0.007261
Saving session checkpoints\config5\model.ckpt-14500
Batch 1500

In [6]:
run1()

Restored session checkpoints\config1\model.ckpt-5000
Batch 5500:
--> Train loss = 0.007463
--> Valid loss = 0.007680
Saving session checkpoints\config1\model.ckpt-5500
Batch 6000:
--> Train loss = 0.007401
--> Valid loss = 0.007639
Saving session checkpoints\config1\model.ckpt-6000
Batch 6500:
--> Train loss = 0.007342
--> Valid loss = 0.007593
Saving session checkpoints\config1\model.ckpt-6500
Batch 7000:
--> Train loss = 0.007267
--> Valid loss = 0.007588
Saving session checkpoints\config1\model.ckpt-7000
Batch 7500:
--> Train loss = 0.007179
--> Valid loss = 0.007587
Saving session checkpoints\config1\model.ckpt-7500
Batch 8000:
--> Train loss = 0.007085
--> Valid loss = 0.007499
Saving session checkpoints\config1\model.ckpt-8000
Batch 8500:
--> Train loss = 0.007006
--> Valid loss = 0.007590
Saving session checkpoints\config1\model.ckpt-8500
Batch 9000:
--> Train loss = 0.006934
--> Valid loss = 0.007610
Saving session checkpoints\config1\model.ckpt-9000
Batch 9500:
--> Train loss 